In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from scipy.signal import correlate
import os
import scipy as sp
import scipy.optimize

from isingpackage import*

%matplotlib inline

In [2]:
def heat_capacity_supervised(N, T, measurements=2000, inverse=False, max_tries=50, M_flip_thresh = 0.4):

    array = spin_array(N=N, randomseed = 123)

    eq_wait = 1000

    for i in range(eq_wait):
        array.update_array(T=T)

    tries = 0

    while True:
        E_list = []
        M_list = []

        for i in range(measurements):
            array.update_array(T=T)
            E_list.append(array.get_E())
            M_list.append(array.get_M_mean())

        if T>2:
            C = np.var(E_list)/(T**2)
            break
        
        #M didn't flip throughout the measurements
        elif np.abs(M_list).min() > M_flip_thresh:
            C = np.var(E_list)/(T**2)
            break

        tries +=1
        #too many tries, lowers the difficulty
        if tries == max_tries:
            measurements = int(measurements*0.9)
            tries = 0
        


    if inverse:
        return 1/C
    else:
        return C


In [8]:
def heat_capacity(T, N, measurements=2000, inverse=False, max_tries=50, M_flip_thresh = 0.4):

    array = spin_array(N=N, randomseed = 123)

    eq_wait = 1000

    for i in range(eq_wait):
        array.update_array(T=T)


    E_list = []
    M_list = []

    for i in range(measurements):
        array.update_array(T=T)
        E_list.append(array.get_E())
        M_list.append(array.get_M_mean())

    C = np.var(E_list)/(T**2)

    if inverse:
        return 1/C
    else:
        return C


In [25]:
def heat_capacity_inverse(T, N, measurements=2000, repeat=1):

    C_list = []

    for j in range(repeat):
        E_list = []
        M_list = []

        array = spin_array(N=N, randomseed = j)

        eq_wait = 1000

        for i in range(eq_wait):
            array.update_array(T=T)



        for i in range(measurements):
            
            array.update_array(T=T)
            E_list.append(array.get_E())
            M_list.append(array.get_M_mean())

        C = np.var(E_list)/(T**2)
        C_list.append(C)

    C_mean = np.mean(C_list)

    return 1/C_mean


In [4]:
Ns = [5,10,15,20,25,32]

for N in Ns:

    print('N={}'.format(N))
    
    measurements=1000

    repeat = 20

    string = ''

    Ts = np.linspace(1,3, 100)

    for T in tqdm(Ts):

        string += '{} '.format(T)

        for i in range(repeat):

            C = heat_capacity(N=N, T=T, measurements=measurements, max_tries=100)

            string = string + ', ' + str(C) 

        string += '\n'

        with open('measurements\\C_{}.csv'.format(N), 'w') as f:
            f.write(string)

N=5


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [28]:
Tc = scipy.optimize.golden(heat_capacity_inverse, args=(128,500,10), brack=(2,2.5,3))
Tc

2.253288905264518